In [1]:
from datetime import datetime
import pandas as pd
import numpy as np

from src.data_io.ascat_readers import AscatH121DynSlope
from src.data_io.ascat_readers import read_multiple_ds
from src.data_io.lsm_readers import Era5, Era5Land
from scipy.optimize import least_squares
from functools import partial
import multiprocessing as mp
from pathlib import Path
import time
from src.examples.subsurface_scattering_parameters import min_max_normalize, normalize_from_40, calculate_c_sigma, sigfunc, performfit

%matplotlib widget

In [2]:
def read_data1(gpi_ascat, ascat, era5land, era5):
    lon, lat = ascat.grid.gpi2lonlat(gpi_ascat)
    
    df = read_multiple_ds(
     loc=(lon, lat),
     ascat=ascat,
     era5=era5,
     era5land=era5land,
     ref_ds="ascat",
     print_gpis=False)

    not_valid = (df["stl1_era5"] < 0) | (df["sd"] > 0)
    df.loc[:, "sm_valid"] = ~not_valid

    df_ts = df.loc[df["sm_valid"]]

    return df_ts

In [3]:
def process_gpi(gpi_ascat, ascat, era5land, era5):
    lon, lat = ascat.grid.gpi2lonlat(gpi_ascat)

    era5_data = era5.read(lon, lat)

    df_ts = read_data1(gpi_ascat, ascat, era5land, era5)

    # Resample ERA5-Land data to daily frequency and
    # filter out snow-covered and frozen soil periods
    daily_era = era5_data.resample('D').mean()
    daily_era['snow_free'] = (
            daily_era['sd'].rolling(window=16,
                                    center=True,
                                    min_periods=8).max() == 0)
    filtered_era = daily_era[(daily_era['snow_free']) &
                             (daily_era['t2m'] > 2)]
    
    # Exclude all data from days with snow cover/frozen soil
    df_ts = df_ts[df_ts.index.floor('D').isin(filtered_era.index)]
    
    # Filter out invalid ASCAT flags
    df_ts = df_ts[(df_ts.processing_flag == 0) &
                  (df_ts.surface_soil_moisture_noise < 50) &
                  (df_ts.surface_soil_moisture_sensitivity > 1)]

    # Check if the dataset meets the criteria for further analysis
    if len(df_ts) < 500 or df_ts.index.to_series().dt.month.nunique() < 6:
        # Not enough data points available, return NaNs
        params = [np.nan] * 4
        theta_turn = np.nan
        sigma_dry = np.nan
        print("not enough data points")
    else:
        df_ts["gpi_ascat"] = gpi_ascat
        df_list.append(df_ts)
        # Convert ERA5 model soil moisture to an index
        df_ts['swvl1_era5land_idx'] = min_max_normalize(
            df_ts['swvl1_era5land'])
    
        # Extrapolate ASCAT backscatter to 20 degrees incidence angle
        df_ts['backscatter20'] = normalize_from_40(
            df_ts['backscatter40'],
            inc_norm,
            df_ts['slope40'],
            df_ts['curvature40'])
    
        # Convert ASCAT backscatter to linear units for model fitting
        df_ts['backscatter20_lin'] = (10 **
                                      (df_ts['backscatter20'] / 10.))
    
        # Provide start values and bounds for the fit
        start_vals = [i[0] for i in start_bounds]
        bounds = np.array([i[1] for i in start_bounds]).T
    
        # Calculate constant sigma for the year
        c_sigma = calculate_c_sigma(df_ts,
                                    sig_col='backscatter20_lin',
                                    sm_col='swvl1_era5land_idx')
    
        # Fit model parameters
        params = performfit(dataset=df_ts,
                            start_vals=start_vals,
                            bounds=bounds,
                            dB=False,
                            c_sigma=c_sigma,
                            fit_q=fit_q,
                            sig_col='backscatter20_lin',
                            sm_col='swvl1_era5land_idx')
    
        param_A = params[0]
        param_B = params[1]
        param_C = params[2]
        param_D = params[3]
    
        # Calculate turning point
        theta_turn = (1 / (param_B + param_D)) * np.log(
            (param_C * param_D) / (param_A * param_B))
    
        # Calculate backscatter at dry conditions
        sigma_dry = sigfunc(A=param_A,
                            B=param_B,
                            C=param_C,
                            D=param_D,
                            V=c_sigma,
                            mv=0,
                            dB=False)
    return gpi_ascat, lon, lat, params, theta_turn, sigma_dry

In [4]:
# Define ASCAT cells that will be processed
cells = [1248]

# Number of cpus used
n_cpus = mp.cpu_count() - 1

# Indicator what parameters should be fitted (True)
# or used as fixed-values (False)
fit_q = [True, True, True, True]

# Start values and boundaries: [start, [min, max]]
# for A, B, C and D
start_bounds = [[0.05, [0.001, 0.3]],
                [5., [0.0, 30.0]],
                [0.15, [0.0, 0.3]],
                [5., [0.0, 30.0]]]

# New incidence angle to normalize sig40 data to
# to minimize vegetation effects
inc_norm = 20

# If resulting parameter estimates should be stored
save = False
# Get the parent directory
parent_dir = Path('__file__').resolve().parent
# Define the new folder name
out_dir = parent_dir / "output"
# Create the folder if it doesn't exist
out_dir.mkdir(exist_ok=True)
# Create file name
out_file = out_dir / "params.csv"

# Create ASCAT and ERA5-Land data objects
ascat = AscatH121DynSlope(read_bulk=True)
era5 = Era5(read_bulk=True)
era5land = Era5Land(read_bulk=True)

# Get GPIs, lats and lons of cell IDs
gpis_ascat = np.array([])

for cell in cells:
    gpis_ascat = np.append(gpis_ascat, ascat.grid.grid_points_for_cell(cell)[0])

df_list=[]
res = []
for gpi in gpis_ascat:
    print(gpi)
    res.append(process_gpi(gpi, ascat, era5land, era5))

820882.0
820971.0
821026.0
821115.0
821204.0
821259.0
821348.0
821437.0
821492.0
821581.0
821636.0
821725.0
821814.0
821869.0
821958.0
822013.0
822102.0
822191.0
822246.0
822335.0
822424.0
822479.0
822568.0
822623.0
822712.0
822801.0
822856.0
822945.0
823034.0
823089.0
823178.0
823233.0
823322.0
823411.0
823466.0
823555.0
823610.0
823699.0
823788.0
823843.0
823932.0
824021.0
824076.0
824165.0
824220.0
824309.0
824398.0
824453.0
824542.0
824686.0
824775.0
824830.0
824919.0
825008.0
825063.0
825152.0
825207.0
825296.0
825385.0
825440.0
825529.0
825618.0
825673.0
825762.0
825817.0
825906.0
825995.0
826050.0
826139.0
826194.0
826283.0
826372.0
826427.0
826516.0
826605.0
826660.0
826749.0
826804.0
826893.0
826982.0
827037.0
827126.0
827215.0
827270.0
827359.0
827414.0
827503.0
827592.0
827647.0
827736.0
827791.0
827880.0
827969.0
828024.0
828113.0
828202.0
828257.0
828346.0
828401.0
828490.0
828579.0
828634.0
828723.0
828867.0
828956.0
829011.0
829100.0
829189.0
829244.0
829333.0
829388.0
8

[(np.float64(820882.0),
  np.float32(-8.368923),
  np.float32(30.001425),
  array([1.00000000e-03, 4.06598433e+00, 5.69222768e-02, 3.00000000e+01]),
  np.float64(0.17730966569890153),
  np.float64(0.3235222987316144)),
 (np.float64(820971.0),
  np.float32(-6.5599236),
  np.float32(30.005001),
  array([1.00000003e-03, 4.00282810e+00, 1.00606633e-02, 2.99999919e+01]),
  np.float64(0.12713148637590194),
  np.float64(0.18551815868278332)),
 (np.float64(821026.0),
  np.float32(-9.486946),
  np.float32(30.00721),
  array([1.00000000e-03, 4.28033561e+00, 9.16249577e-24, 3.00000000e+01]),
  np.float64(-1.2891356398926295),
  np.float64(0.12204239191394081)),
 (np.float64(821115.0),
  np.float32(-7.677947),
  np.float32(30.010784),
  array([1.00000000e-03, 3.06877020e+00, 4.32739067e-02, 2.02221631e+01]),
  np.float64(0.24271471034840364),
  np.float64(0.27173290864042865)),
 (np.float64(821204.0),
  np.float32(-5.8689475),
  np.float32(30.01436),
  array([1.00000003e-03, 4.63892916e+00, 5.3484

In [103]:
df_ts["target"] = np.where(df_ts["swvl1_era5"] < abs(theta_turn), 0 , 1)

In [10]:
df = pd.concat(df_list)

In [12]:
res

[(np.float64(820882.0),
  np.float32(-8.368923),
  np.float32(30.001425),
  array([1.00000000e-03, 4.06598433e+00, 5.69222768e-02, 3.00000000e+01]),
  np.float64(0.17730966569890153),
  np.float64(0.3235222987316144)),
 (np.float64(820971.0),
  np.float32(-6.5599236),
  np.float32(30.005001),
  array([1.00000003e-03, 4.00282810e+00, 1.00606633e-02, 2.99999919e+01]),
  np.float64(0.12713148637590194),
  np.float64(0.18551815868278332)),
 (np.float64(821026.0),
  np.float32(-9.486946),
  np.float32(30.00721),
  array([1.00000000e-03, 4.28033561e+00, 9.16249577e-24, 3.00000000e+01]),
  np.float64(-1.2891356398926295),
  np.float64(0.12204239191394081)),
 (np.float64(821115.0),
  np.float32(-7.677947),
  np.float32(30.010784),
  array([1.00000000e-03, 3.06877020e+00, 4.32739067e-02, 2.02221631e+01]),
  np.float64(0.24271471034840364),
  np.float64(0.27173290864042865)),
 (np.float64(821204.0),
  np.float32(-5.8689475),
  np.float32(30.01436),
  array([1.00000003e-03, 4.63892916e+00, 5.3484

In [23]:
df = pd.concat(df_list)
df["soil_cat"] = None
for x in res:
    if (x[4] < 0):
        df.loc[df["gpi_ascat"] == x[0], 'soil_cat'] = 'wet'
    elif (x[4] > 1):
        df.loc[df["gpi_ascat"] == x[0], 'soil_cat'] = 'dry'
    elif ((x[4] > 0) & (x[4] < 1)):
        df.loc[df["gpi_ascat"] == x[0], 'soil_cat'] = 'mixed'

In [40]:
df1 = df.copy()
df1 = df1[df1["soil_cat"] == "mixed"]
df1["target"] = None

helpy = pd.DataFrame(res, columns=["gpi_ascat", "lon", "lat", "params", "theta_turn", "sigma_dry"])
df1 = df1.join(helpy.set_index('gpi_ascat'), on='gpi_ascat')

df1["target"] = np.where(df1["swvl1_era5land"] < df1["theta_turn"], 0, 1)
df1 = df1.drop(['sat_id', 'as_des_pass', 'swath_indicator', 'surface_soil_moisture',
       'surface_soil_moisture_noise','slope40_noise', 'curvature40_noise', 'backscatter40_noise', 'backscatter40_flag', 'dry40', 'dry40_noise',
       'wet40', 'wet40_noise', 'surface_soil_moisture_sensitivity',
       'processing_flag', 'correction_flag', 'surface_state_flag', 'lon', 'lat', 'params', 'theta_turn',
       'sigma_dry'], axis=1)


df1.to_csv('ascat_era5_era5land.csv')


           